# Getting Started with LEIP Recipes
An End to End tutorial
---

In this notebook we will run through an example customer use case end to end.
Starting from just having an annotated dataset, we will get some recipes from the Golden Recipes database that are likely to perform well on our data, train them, and check the actual performance of our trained models in framework before exporting the best one. 
Then we will optimize and compile our models for our example hardware: a CUDA enabled NVIDIA device.

Our example data today is Road Sign Detection from kaggle.

The steps we will follow in this tutorial are:
1. Download the Road Sign Detection dataset from kaggle, and get familiar with it.
2. Select some candidate recipes: 
    - Explore the Golden Recipe Volumes available today.
    - Select a volume of Golden Recipes that is most likely to be a good match for our dataset.
    - Pick 10 candidate recipes to try on our Road Sign Detection data.
3. Train the candidate recipes on the Road Sign Detection data.
4. Evaluate the models trained using the candidate recipes, select the best performer, and export it.
5. Optimize the best performer and compile it for deployment.
6. Evaluate the optimized model to ensure enough accuracy has been preserved.

## Set Up

If you have not done so already, please ensure your environment is set up according to [Setting up a LEIP Environment](../environment/README.md). This will ensure that while executing this notebook the APIs of both the [Application Framework](https://leipdocs.latentai.io/af/latest/content/) and [Compiler Framework](https://leipdocs.latentai.io/cf/latest/content/) are accessible.

## Initialization

Here we initialize some variables and ensure `leip_client` is able to connect to the server side container. A green checkbox printed after `Checking connection` signifies a successful connection.

In [ ]:
import os
from pathlib import Path

from leip_client import Leip
from rich import print

# Create a LEIP instance
leip = Leip.load_instance()
host = os.environ.get("LEIP_CF_HOST", "http://127.0.0.1:8080")

# Configure connection to CF container
leip.create_profile("default", host, os.environ["LICENSE_KEY"])
leip.check_connection(silent=False)

# Initialize shared workspace volume
assert os.environ.get("LEIP_WORKSPACE"), "Make sure your environment variable LEIP_WORKSPACE is set according to the environment setup instructions."
workspace = Path(os.environ.get("LEIP_WORKSPACE"))

## Step 1: Get the dataset

Download the Road Sign Detection dataset from kaggle, and get familiar with it: open [the dataset page on Kaggle](https://www.kaggle.com/datasets/andrewmvd/road-sign-detection) and click "download (229mb)". This may require signing in to Kaggle using your own credentials. Then unzip the downloaded file to `$LEIP_WORKSPACE/datasets/kaggle/road-sign-data`.

For your convenience, we have mirrored the dataset so that you can download and unzip by running this:

In [ ]:
import requests
import shutil

dataset_dir = workspace / "datasets" / "kaggle" / "road-sign-data"

if not dataset_dir.exists():
    mirror_url = "https://s3.us-west-1.amazonaws.com/leip-showcase.latentai.io/recipes/andrewmvd_road-sign-detection.zip"
    local_file = Path("andrewmvd_road-sign-detection.zip")

    print("Downloading dataset...")

    response = requests.get(mirror_url)
    with open(local_file, "wb") as f:
        f.write(response.content)

    shutil.unpack_archive(local_file, dataset_dir)

    print(f"Dataset now available at {dataset_dir}")

To design recipes, we need ingredients. Ingredients live in the pantry. A pantry can have all sorts of ingredients, from models to datasets, optimizers and learning rate schedulers, and even training-aware quantization techniques. Let's download a pantry full of useful ingredients to start:

In [ ]:
from leip_recipe_designer import Pantry
pantry = Pantry.download()

To ingest our Road Sign Data, we need a data_generator ingredient for it. A data_generator ingredient tells the execution where from and how to ingest the dataset.

The `new_pascal_data_generator` helper function will help us create the data_generator ingredient for our Road Sign Data, so we can use it in our recipes. 

_Tip: This function won't execute any task; instead, its whole purpose is to help us generate the ingestion ingredient correctly. Once generated, the ingredient will be added to a recipe, and the `visualize_data` task will be run on that recipe._

Make some observations about the folder structure and format of the data, so we can properly create the data_generator ingredient:
- The data is in the **Pascal VOC format**, so we will use the `new_pascal_data_generator` helper function.
- The annotations are in XML files under a folder called `annotations` and images under an `images` folder. We will enter that information for `annotations_dir` and `images_dir`.
- The dataset appears to **not be pre-split** into a training and validation set. We will set `is_split = False`.
- There are **4 object classes** in our data. We will set `nclasses = 4`.
- The dataset contains a *total of 877 samples*. Images have low resolutions with varying aspect ratios; some landscape and some portrait oriented. This information does not go into the data_generator, but it may help us find the most appropriate golden recipes in the next steps.

In [ ]:
from leip_recipe_designer.helpers.data import new_pascal_data_generator

road_sign_data = new_pascal_data_generator(
    pantry=pantry,
    root_path=str(dataset_dir),
    images_dir="images",
    annotations_dir="annotations",
    nclasses=4,
    is_split=False,
    trainval_split_ratio=0.80,
    dataset_name="road-sign-data"
)

Tasks such as `visualize_data` run on recipes. All we have at this point is one ingredient. Let's start a blank recipe, fill it with only the essentials, and add this data_generator ingredient to it.

In [28]:
from leip_recipe_designer.create import empty_detection_recipe
my_temporary_recipe = empty_detection_recipe(pantry=pantry)
my_temporary_recipe.fill_empty_recursively()

from leip_recipe_designer.helpers.data import replace_data_generator
replace_data_generator(recipe=my_temporary_recipe, data=road_sign_data)

In [ ]:
from leip_recipe_designer import tasks
vizdata_outputs = tasks.visualize_data(my_temporary_recipe)

All tasks return a dictionary with any useful outputs of the task. In the case of the `visualize_data` task, the dictionary contains the directories where the sample images were stored. Let's print it out:

In [ ]:
print(vizdata_outputs)

We can write a little function to display the saved images in this Jupyter notebook.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
%matplotlib inline

def display_helper(images_path, count=4, image_extension="jpeg"):
    fig = plt.figure(figsize=(20, 10), facecolor='w')
    columns = 4
    rows = math.ceil(count / columns)
    for idx, path in zip(range(rows * columns), Path(images_path).rglob(str("*."+image_extension))):
        img = mpimg.imread(path)
        fig.add_subplot(rows, columns, idx + 1)
        plt.imshow(img)
        plt.axis("off")

print("Some sample training images:")
display_helper(vizdata_outputs["vizdata.output_directory.train"], count=8)

In [ ]:
print("Some sample validation images:")
display_helper(vizdata_outputs["vizdata.output_directory.val"], count=8)

## Step 2: Select candidate recipes

Golden Recipe volumes are collections of recipes that have proven to perform well on a particular dataset. Latent AI has computers churning away discovering better recipes for more and more diverse datasets.

Let's list the available volumes as of today.

In [ ]:
from leip_recipe_designer import GoldenVolumes
goldenvolumes = GoldenVolumes()
volumes = goldenvolumes.list_volumes_from_zoo()

Each of the above names correspond to a Golden Recipe Volume: a set of recipes that performed well on a particular dataset. We can explore the datasets used to train each of the volumes and try to find the one that comes closest to our data. 

In our case, our data is simple, with less than 1000 samples, and contains outdoor images of sparse objects with resolutions around the 300x400 and 400x300 range.

If any of the volumes strike you as having data that is really similar to yours, go ahead and start with that. Note that data similarity is not a clearly defined concept, so pick one volume to start, and iterate from there. For more information read [How to Pick the Best GRDB Volume to Start](/af/latest/content/getting-started/how_to/#pick-the-best-grdb-volume-to-start).


One of the GRDBs is a car detection data. It is a very small dataset of objects with simple shapes. Perhaps that is a good place to start.

In [ ]:
volumes["carsimple"]

Good performance can mean different things to different people: task accuracy is a good measure of performance, but maybe you care more about fast training than getting the most accuracy possible, or perhaps having a small model that runs inference very quickly in your device of interest is imperative for your application.

Because of this, Golden Recipe volumes are structured as Pandas dataframes that you can query and sort according to your requirements. 

In [ ]:
df = volumes["carsimple"].get_golden_df()
df

In our example, we will keep things simple: we want candidate recipes that do not sacrifice too much accuracy and are also likely to run fast once compiled (based on the number of Multiply-Accumulate Operations during inference)

In [ ]:
threshold = df["if_task_metric"].quantile(0.75)
filtered_df = df[df["if_task_metric"] >= threshold]
sorted_df = filtered_df.sort_values("if_inf_macs")
sorted_df.head()

Let's retrieve the SPPR (Serialized Packaged Portable Recipe) from each of the results. To deserialize, use `from_sppr`.
We don't have unlimited amounts of time to train, so we will limit our list of candidate recipes for training to the best 4. 

Once you deserialize, training the candidate recipes only requires swapping the data ingredient of each of the candidate recipes with your data.

In [ ]:
from leip_recipe_designer.create import from_sppr

optimal_models = ["GRDBCAR-9", "GRDBCAR-213", "GRDBCAR-66", "GRDBCAR-12", "GRDBCAR-166"]

# Extract the recipes
candidate_recipes = {}
for recipe_id in optimal_models:
    sppr = sorted_df.query(f"id == '{recipe_id}'").iloc[0]["sppr"]
    candidate_recipes[recipe_id] = from_sppr(sppr, pantry, allow_upgrade=True)
print(f"We have collected the {len(candidate_recipes)} recipes with lowest MACs that meet our task metric performance criteria")

## Step 3: Train the candidate recipes

_Tip: no need to wait for training!_

Training these recipes until convergence will take several minutes: an NVIDIA RTX A4500 GPU took about 40 minutes to train these 4 models. We recognize you may want to continue with the tutorial without having to wait until convergence, so we've added a few lines below to limit training time to only one tenth of an epoch. After this short training showcase, a few lines of code below will download the trained checkpoints, so you can move to the next step of evaluating the recipes.

**If you prefer to wait until training converges instead of downloading the checkpoints, remove `recipe["train.num_epochs"] = 1; recipe["trainer.train_batches_percentage"] = 0.1`. These lines limit training to just 10% of one epoch.**

In [ ]:
from leip_recipe_designer.helpers.data import replace_data_generator
from leip_recipe_designer import tasks

train_outputs = {}
for recipe_id, recipe in candidate_recipes.items():
    # it is necessary to add a logger to our run. Below we are adding a default local log.
    # If you use a different logger such as Weights and Biases or Neptune,
    # visit our documentation for instructions on how to add it to the recipe
    recipe.assign_ingredients('loggers', {"my_local_training_log": "Tensorboard"})

    # in place change in the recipe. Swaps the recipe's original training data with your data
    replace_data_generator(recipe, road_sign_data)
    try:
        # change the mosaic augmentation sampling scheme to tolerate different input resolution
        recipe["data_generator.composite.mosaic.sampling_scheme"] = "preserve_spatial"
    except KeyError:
        pass

    # THE BELOW LINE CUTS TRAINING SHORT, FOR THE SAKE OF TIME. IF YOU HAVE TIME TO WAIT UNTIL CONVERGENCE,
    # COMMENT THE LINE BELOW AND THE MODEL WILL STOP AUTOMATICALLY ONCE IT'S DONE TRAINING
    recipe["train.num_epochs"] = 1
    recipe["trainer.train_batches_percentage"] = 0.1

    # This is a completely optional step. Since we are training multiple recipes, I will use the recipe ID to identify the artifacts generated by this recipe
    recipe["experiment.name"] = recipe_id

    print(f"\n\nTraining recipe {recipe_id}")
    train_output = tasks.train(recipe)
    train_outputs[recipe_id] = train_output

    # After training is finished for a recipe, add the checkpoint path to the recipe, so it can be used by the evaluate and export tasks below
    recipe["model.checkpoint"] = str(train_output["best_model_path"])

#### Download the checkpoints

After waiting for the short training to complete above, you will have training outputs. As mentioned earlier in this tutorial, all tasks return a dictionary pointing to any outputs generated from the task.

Unless you commented out the specified lines, you only trained for 10% of an epoch, so we don't expect your model to have learned much. The cells below will download and extract the checkpoints that were trained in Latent AI servers, and modify your training outputs to point to the downloaded checkpoints instead.

In [ ]:
# Replace these values with the URL of the file you want to download
file_url = "https://s3.us-west-1.amazonaws.com/leip-showcase.latentai.io/recipes/tutorials/GettingStartedCheckpoints.zip"

# Specify the local directory to save the downloaded and extracted files
local_directory = Path("downloaded_checkpoints")

# Create the local directory if it doesn't exist
local_directory.mkdir(exist_ok=True, parents=True)

# Path to save the downloaded file
zip_file_path = local_directory / "GettingStartedCheckpoints.zip"

# Download the file
response = requests.get(file_url, stream=True)
if response.status_code == 200:
    with open(zip_file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# Extract the contents of the zip file
shutil.unpack_archive(zip_file_path, local_directory)

print("Downloaded and extracted files to:", local_directory)

The cell below replaces the trained checkpoints with the downloaded ones.

In [ ]:
file_paths = []
for root, dirs, files in os.walk(local_directory):
    for filename in files:
        file_path = os.path.abspath(os.path.join(root, filename))
        file_paths.append(file_path)

for recipe_id, recipe in candidate_recipes.items():
    for file_path in file_paths:
        if recipe_id in file_path and "ckpt" in file_path:
            recipe["model.checkpoint"] = file_path
            print("Added the downloaded checkpoint from", file_path, "to the correct recipe", recipe_id)

## Step 4: Pick a winner

Evaluate the models trained using the candidate recipes, and select the best performer.

In [ ]:
evaluate_outputs = {}
for recipe_id, recipe in candidate_recipes.items():
    eval_output = tasks.evaluate(recipe)
    evaluate_outputs[recipe_id] = eval_output

for recipe_id, scores in evaluate_outputs.items():
    print("Recipe with ID", recipe_id, "has a Mean Average Precision score (averaged over IoU Thresholds 0.50:.95:0.05) of", scores["evaluate.metric_single"])

**The highest task accuracy was obtained by the recipe with ID GRDBCAR-213**

We can visualize its predictions on the data using the helper we defined earlier in this tutorial. This is a completely optional step to ensure things look good before we export the model.

In [ ]:
best_recipe = candidate_recipes["GRDBCAR-213"]
predict_output = tasks.visualize_predictions(best_recipe)

In [ ]:
display_helper(predict_output["predict.output_directory"], count=16)

Now we are ready to export our recipe as a traced artifact.

In [ ]:
model_path = workspace / "exported"
best_recipe["export.output_directory"] = str(model_path)
export_output = tasks.export_model(best_recipe)
print(f"Your model has been saved to disk under {export_output['export.model_path']}.\nYou can now move forward with optimizing your model using the LEIP Compiler Framework.")

## Step 5: Optimize Your Recipe

Next we will optimize the model exported in the last section. We will use the Pipeline API to group together a couple of subtasks into one batched execution. One subtask will optimize and compile the model into `int8` for its target, and the other subtask will compile to `float32`.

We will set a few variables that will be used below.

In [ ]:
dataset_path = workspace / "datasets" / "kaggle" / "road-sign-data"
target = "cuda"
target_host = "llvm -mcpu=skylake"

Next we will run LEIP Pipeline. The `model_options`, `compress_options` and `compile_options` together comprise the various options needed to optimize and compile the model. Refer to the [Compiler Framework documentation](https://leipdocs.latentai.io/cf/latest/content/) for more information on the available options.

The optimize step requires a *representative dataset* which is used for calibration during the quantization of activations in the model. A good rule of thumb for creating a representative dataset is to include at least one image from each output class in the dataset. Often though, even a few input samples from the dataset are enough to get good quantized accuracy for the model. Other times the success of the quantization can be very sensitive to the representative dataset chosen. So you might want to experiment in this area. We will choose a simple representative dataset of 10 randomly selected images for this tutorial.

In [ ]:
rep_dataset_paths = [
    dataset_dir / "images" / "road114.png",
    dataset_dir / "images" / "road14.png",
    dataset_dir / "images" / "road214.png",
    dataset_dir / "images" / "road314.png",
    dataset_dir / "images" / "road414.png",
    dataset_dir / "images" / "road514.png",
    dataset_dir / "images" / "road614.png",
    dataset_dir / "images" / "road714.png",
    dataset_dir / "images" / "road814.png",
    dataset_dir / "images" / "road77.png"
]
rep_dataset_items = [
    str(path) for path in rep_dataset_paths
]
rep_dataset_file = workspace / "rep_dataset.txt"
rep_dataset_file.write_text("\n".join(rep_dataset_items))

The following cell runs the pipeline. Please note that the pipeline might take a few minutes to run the optimization step.

In [ ]:
from leip_client import (
    TaskModelOptions,
    CompressOptions,
    CompileOptions,
    PipelineTask,
    PipelineInnerTask,
    CompilePipelineOptions,
    OptimizePipelineOptions,
    OutputOptions,
)

model_options = TaskModelOptions(
    path=model_path,
    task_family="detection",
)

compress_options = CompressOptions(
    rep_dataset=rep_dataset_file,
    quantizer="symmetricpc",
)

compile_options = CompileOptions(
    target=target,
    target_host=target_host,
)

pipeline_config = PipelineTask(
    name="RecipePipeline",
    tasks=[
        PipelineInnerTask(
            name="Int8",
            optimize=OptimizePipelineOptions(
                model=model_options,
                compress=compress_options,
                compile=compile_options,
            ),
        ),
        PipelineInnerTask(
            name="Float32",
            compile=CompilePipelineOptions(
                model=model_options,
                target=compile_options.target,
                target_host=compile_options.target_host,
            ),
        ),
    ],
    output=OutputOptions(path=workspace)
)

print("Optimizing model...")
pipeline_job = pipeline_config.run()
print(pipeline_job)

## Step 6: Evaluate

Now we will use the Evaluate API to run an evaluation of the optimized model on the server. This will return a mean average precision (mAP) for how well the optimized model does on the validation set of the dataset in our detection task. Note that the mAP score might not be as good as the scores above when evaluated in-framework, but it should not have dropped significantly.

Refer to [the LEIP Evaluate module](https://leipdocs.latentai.io/cf/latest/content/modules/evaluate/) for more information on using LEIP Evaluate for evaluating the accuracy of a model throughout the various stages in your pipeline.


In [ ]:
from leip_client import (
    DatasetOptions,
    PascalVOCOptions,
    EvaluateTask,
)
from leip_zoo import DatasetSchema

int8_model = workspace / "Int8-optimize"

best_recipe.assign_ingredients("export_data", "COCO")
best_recipe["export_data.subset"] = "val"
best_recipe["export_data.save_directory"] = str(workspace / "datasets" / "kaggle")
export_data_output = tasks.export_data(best_recipe)
schema_file = Path(export_data_output["export_data.output_directory"]) / "dataset_schema_val.yaml"
dataset_options = DatasetSchema.load(schema_file, options={"root_dir": schema_file.parent}).options

evaluate_task = EvaluateTask(
    model=TaskModelOptions(path=int8_model),
    dataset=dataset_options,
)

print("Evaluating optimized int8 model...")
model_eval_job = evaluate_task.run()
print(model_eval_job)
mAP = model_eval_job.results["scoring"]["mAP"]["0.5:0.95:0.05"]
print(f"int8 mAP: {mAP}")

## Next Steps

Now that we have optimized and compiled our model (using LEIP Optimize) and evaluated how well the model performs (using LEIP Evaluate) from a host environment, we are ready to deploy the model on an edge device (`cuda` hosted by an `llvm -mcpu=skylake` for this tutorial).

To streamline the process of transferring the optimized model for deployment, we'll assemble all model-related files into a single tar file:

In [ ]:
import tarfile

with tarfile.open(workspace / "int8_model.tar.gz", "w:gz") as tar:
    for file in int8_model.glob("*"):
        tar.add(file, arcname=file.name)

We will also copy an image to test on target device:

In [ ]:
cp $LEIP_WORKSPACE/datasets/kaggle/road-sign-data/images/road314.png $LEIP_WORKSPACE/road314.png

We will also create a labels file for visualization on the target:

In [ ]:
values=$(jq -r 'values | .[]' < "$LEIP_WORKSPACE/datasets/kaggle/road-sign-data/pascal_label_map.json")
echo "background" > "$LEIP_WORKSPACE/roadsign.txt" # leave an empty line for index 0 which maps to background, overwrite an existing file
echo "$values" | tr '\n' '\n' >> "$LEIP_WORKSPACE/roadsign.txt" # add each label as a new line after the empty line

Following are the artifacts from our host enviroment that we would like to take to our deployment environment.
1. Compiled model and metadata: $LEIP_WORKSPACE/int8_model.tar.gz
2. Test images: $LEIP_WORKSPACE/road314.png  
3. Application artifacts: $LEIP_WORKSPACE/roadsign.txt

Once we have our deployment environment ready and artifacts transfered to the deployment device, we can deploy our compiled model with Python or C++. Since the models we selected are detection models we will use the detectors example. [Detectors example implementations](https://github.com/latentai/example-applications/tree/v3.0.1/detectors) provide details on how to deploy along with its implementation details and options to customize.

We can use options provided in the detectors example to deploy the compiled model and visualize on the image we selected. Parameters used in the evaluation environment are available in `$INT8_MODEL/model_schema.yaml`. However, we can modify these parameters to adjust the visualization to display few useful detections on our selected image. Selection of these parameters have accuracy and latency implications. (For example, we ran for 10 iterations for steady state latency numbers, we used `maximum_detections` = 5, `confidence_threshold` = 0.6, `iou_threshold` = 0.45 to get most likely detections fast.)

Deployment example applications are built using the [Runtime Framework](https://leipdocs.latentai.io/rf/content/) where you can find details about the API and its usage.